<a href="https://colab.research.google.com/github/Orshikhbayar/ml/blob/main/Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import csv
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from datetime import datetime, timedelta, timezone
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import ipywidgets as widgets
from IPython.display import display
import yfinance as yf

# Download the required resource for sentiment analysis
nltk.download('vader_lexicon')

# Load the NLTK sentiment intensity analyzer
sia = SentimentIntensityAnalyzer()

# Fetch available tickers from Yahoo Finance
tickers = ['BTC-USD', 'PXP-USD', 'XRP-USD', 'ETH-USD', 'XDC-USD', 'MCH-USD', 'XLM-USD', 'ETC-USD', 'REV-USD', 'DOGE-USD', '1INCH-USD', 'ADA-USD', 'VET-USD', 'VAL-USD', 'HBAR-USD', 'RSR-USD', 'DFI-USD', 'LTC-USD', 'TRX-USD', 'NEO-USD', 'DGB-USD', 'BCH-USD', 'ZRX-USD', 'DOT-USD','PEPE-USD', 'DAI-USD', 'SOL-USD', 'VRA-USD', 'ENJ-USD', 'AVAX-USD', 'GALA-USD', 'ALGO-USD',
           'USDC-USD', 'OMG-USD', 'GRT-USD', 'BAT-USD', 'LINK-USD', 'SC-USD', 'COVN-USD', 'FTM-USD',
           'ZEC-USD', 'DASH-USD', 'WAVES-USD', 'UNI-USD', 'XTZ-USD', 'CRO-USD', 'ATOM-USD', 'CRV-USD',
           'QTUM-USD', 'CELO-USD', 'COMP-USD', 'ETHW-USD', 'AAVE-USD', 'MATIC-USD', 'SNX-USD', 'OCEAN-USD', 'SG-USD', '1ECO-USD', 'JASMY-USD', 'RVN-USD', 'DCR-USD',
           'FIL-USD', 'EOS-USD', 'APE-USD', 'XEM-USD', 'R1-USD', 'ARDX-USD', 'XCN-USD', 'SAND-USD ', 'GXE-USD', 'AKT-USD', 'AXS-USD', 'B2M-USD', 'BAND-USD', 'CEL-USD',
           'DAF-USD', 'GMT-USD', 'HNS-USD', 'IOTX-USD', 'KLAY-USD', 'KOK-USD', 'KSM-USD',
           'MKR-USD', 'MNW-USD', 'MYCE-USD', 'ONT-USD', 'PROS-USD', 'RSV-USD', 'SOLVE-USD', 'TRAC-USD', 'TUSD-USD', 'UMA-USD', 'WBTC-USD' ]

# Create dropdown menu widget for cryptocurrency selection
cryptocurrency_dropdown = widgets.Dropdown(options=tickers, description='Cryptocurrency:')

# Load the data from Yahoo Finance
def load_data(cryptocurrency):
    try:
        data = yf.download(cryptocurrency, period="max")
        data.reset_index(inplace=True)
        data.drop_duplicates(subset='Date', inplace=True)
        data.fillna(0, inplace=True)
        data['date'] = pd.to_datetime(data['Date'])
        return data
    except KeyError:
        print(f"Error: No data available for {cryptocurrency}.")
        return pd.DataFrame()

# Create widgets for user input
investment_amount_widget = widgets.IntSlider(value=1000, min=100, max=10000, step=100, description='Investment Amount ($)')
target_date_widget = widgets.DatePicker(description='Target Date')
predict_button = widgets.Button(description='Predict')

def predict_button_clicked(b):
    # Retrieve user input values
    investment_amount = investment_amount_widget.value
    target_date = target_date_widget.value
    cryptocurrency = cryptocurrency_dropdown.value

    # Load the data for the selected cryptocurrency
    data = load_data(cryptocurrency)

    if data.empty:
        return

    # Prepare the data for prediction
    target_column = 'Close'
    feature_columns = ['Open', 'High', 'Low', 'Volume']

    target = data[target_column].values
    features = data[feature_columns].values

    target_scaler = MinMaxScaler(feature_range=(0, 1))
    feature_scaler = MinMaxScaler(feature_range=(0, 1))

    scaled_target = target_scaler.fit_transform(target.reshape(-1, 1))
    scaled_features = feature_scaler.fit_transform(features)

    train_size = int(len(data) * 0.8)
    train_target = scaled_target[:train_size]
    train_features = scaled_features[:train_size]
    test_target = scaled_target[train_size:]
    test_features = scaled_features[train_size:]

    train_features = np.reshape(train_features, (train_features.shape[0], 1, train_features.shape[1]))
    test_features = np.reshape(test_features, (test_features.shape[0], 1, test_features.shape[1]))

    model = Sequential()
    model.add(LSTM(100, input_shape=(1, len(feature_columns))))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')

    model.fit(train_features, train_target, epochs=50, batch_size=64, verbose=2)

    current_date = datetime.now().date()  # Convert to datetime.date
    last_known_date = data['date'].max().date()  # Convert to datetime.date
    days_to_predict = (target_date - last_known_date).days

    prediction_dates = [target_date]

    predictions = []

    for prediction_date in prediction_dates:
        days_to_predict = (prediction_date - current_date).days
        last_features = test_features[-1:]  # Use the last available features for prediction

        for _ in range(days_to_predict):
            prediction = model.predict(last_features)
            predictions.append(prediction[0][0])

            # Update the features for the next prediction
            last_features[0][0][0] = prediction[0][0]
            last_features = np.array(last_features)

    predicted_prices = target_scaler.inverse_transform(np.array(predictions).reshape(-1, 1))

    print("Predicted Prices:")
    for i, prediction_date in enumerate(prediction_dates):
        print(f"{prediction_date.strftime('%Y-%m-%d')}: {predicted_prices[i][0]:.4f}")

    # Calculate investment value and profit/loss at each time range
    investment_values = []
    profits_losses = []
    investment_date = current_date  # Date of investment

    for i, prediction_date in enumerate(prediction_dates):
        investment_price = target_scaler.inverse_transform(test_target[-1:])[0][0]
        current_price = predicted_prices[i][0]
        investment_value = investment_amount / investment_price * current_price
        investment_values.append(investment_value)

        # Calculate profit/loss
        profit_loss = investment_value - investment_amount
        profits_losses.append(profit_loss)

    print(f"\nInvestment Details (Amount: {investment_amount} USD):")
    for i, prediction_date in enumerate(prediction_dates):
        print(f"{prediction_date.strftime('%Y-%m-%d')}: "
              f"Investment Value: {investment_values[i]:.2f} USD, "
              f"Profit/Loss: {profits_losses[i]:.2f} USD")

    # Perform Sentiment Analysis
    # Calculate the date range
    today = datetime.now().date()
    from_date = (today - timedelta(days=7)).isoformat()
    to_date = today.isoformat()

    # Set your API key and search parameters
    api_key = "ce600ab964514d429fb25b58a005dde6"
    search_query = cryptocurrency
    language = "en"

    # Define the API endpoint URL
    url = f"https://newsapi.org/v2/everything?q={search_query}&from={from_date}&to={to_date}&language={language}&apiKey={api_key}"

    # Send the HTTP request to the API for News data
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON response
        data = response.json()

        # Extract the articles from the response
        articles = data["articles"]

        # Define the CSV filename for News data
        filename = "articles.csv"

        # Open the CSV file in write mode and download the articles
        with open(filename, "w", newline="", encoding="utf-8") as file:
            # Create a CSV writer object
            writer = csv.writer(file)

            # Write the header row
            writer.writerow(["Title", "Description", "Source", "URL", "Published At"])

            # Write each article to a row in the CSV file
            for article in articles:
                title = article["title"]
                description = article["description"]
                source = article["source"]["name"]
                url = article["url"]
                published_at = article["publishedAt"]

                # Write the article data to the CSV file
                writer.writerow([title, description, source, url, published_at])

        print(f"\nArticles have been downloaded and saved to {filename}")

        # Read the articles from the CSV file
        articles = []
        dates = []
        with open(filename, "r", encoding="utf-8") as file:
            reader = csv.reader(file)
            next(reader)  # Skip the header row
            for row in reader:
                title = row[0]
                description = row[1]
                published_at = row[4]
                articles.append(description)
                dates.append(datetime.strptime(published_at, "%Y-%m-%dT%H:%M:%SZ").date())

        # Sort the articles and dates in chronological order
        articles = [a for _, a in sorted(zip(dates, articles))]
        dates.sort()

        # Analyze the sentiment of each article
        sentiments = []
        for article in articles:
            sentiment = sia.polarity_scores(article)
            sentiments.append(sentiment)

        # Merge sentiment scores for each day
        daily_sentiments = {}
        for i, sentiment in enumerate(sentiments):
            date = dates[i]
            if date not in daily_sentiments:
                daily_sentiments[date] = {
                    "compound": [],
                    "positive": [],
                    "negative": [],
                    "neutral": []
                }
            daily_sentiments[date]["compound"].append(sentiment["compound"])
            daily_sentiments[date]["positive"].append(sentiment["pos"])
            daily_sentiments[date]["negative"].append(sentiment["neg"])
            daily_sentiments[date]["neutral"].append(sentiment["neu"])

        # Calculate average sentiment scores for each day
        average_sentiments = {
            "dates": [],
            "compound": [],
            "positive": [],
            "negative": [],
            "neutral": []
        }
        for date, scores in daily_sentiments.items():
            average_compound = sum(scores["compound"]) / len(scores["compound"])
            average_positive = sum(scores["positive"]) / len(scores["positive"])
            average_negative = sum(scores["negative"]) / len(scores["negative"])
            average_neutral = sum(scores["neutral"]) / len(scores["neutral"])
            average_sentiments["dates"].append(date)
            average_sentiments["compound"].append(average_compound)
            average_sentiments["positive"].append(average_positive)
            average_sentiments["negative"].append(average_negative)
            average_sentiments["neutral"].append(average_neutral)

        # Convert average sentiment data to DataFrame
        sentiment_df = pd.DataFrame(average_sentiments)

        # Display sentiment DataFrame
        print("\nSentiment Analysis:")
        print(sentiment_df)

    else:
        print(f"Error: Failed to fetch news data for {cryptocurrency}.")

# Register button click event handler
predict_button.on_click(predict_button_clicked)

# Display the widgets and button
display(cryptocurrency_dropdown, investment_amount_widget, target_date_widget, predict_button)

Dropdown(description='Prediction Mode:', options=('Single Crypto Prediction', 'Multi Crypto Prediction'), valu…

Button(description='Select', style=ButtonStyle())

Dropdown(description='Cryptocurrency:', options=('BTC-USD', 'PXP-USD', 'XRP-USD', 'ETH-USD', 'XDC-USD', 'MCH-U…

IntSlider(value=1000, description='Investment Amount ($)', max=10000, min=100, step=100)

DatePicker(value=None, description='Target Date')

Button(description='Predict', style=ButtonStyle())

IntSlider(value=1000, description='Investment Amount ($)', max=10000, min=100, step=100)

DatePicker(value=None, description='Target Date')

[*********************100%***********************]  1 of 1 completed
Epoch 1/50
21/21 - 7s - loss: 0.0365 - 7s/epoch - 350ms/step
Epoch 2/50
21/21 - 0s - loss: 0.0139 - 78ms/epoch - 4ms/step
Epoch 3/50
21/21 - 0s - loss: 0.0055 - 71ms/epoch - 3ms/step
Epoch 4/50
21/21 - 0s - loss: 0.0010 - 70ms/epoch - 3ms/step
Epoch 5/50
21/21 - 0s - loss: 1.5236e-04 - 71ms/epoch - 3ms/step
Epoch 6/50
21/21 - 0s - loss: 1.4112e-04 - 79ms/epoch - 4ms/step
Epoch 7/50
21/21 - 0s - loss: 1.3074e-04 - 100ms/epoch - 5ms/step
Epoch 8/50
21/21 - 0s - loss: 1.2461e-04 - 100ms/epoch - 5ms/step
Epoch 9/50
21/21 - 0s - loss: 1.2059e-04 - 98ms/epoch - 5ms/step
Epoch 10/50
21/21 - 0s - loss: 1.1606e-04 - 96ms/epoch - 5ms/step
Epoch 11/50
21/21 - 0s - loss: 1.1143e-04 - 92ms/epoch - 4ms/step
Epoch 12/50
21/21 - 0s - loss: 1.0787e-04 - 91ms/epoch - 4ms/step
Epoch 13/50
21/21 - 0s - loss: 1.0359e-04 - 109ms/epoch - 5ms/step
Epoch 14/50
21/21 - 0s - loss: 9.9474e-05 - 97ms/epoch - 5ms/step
Epoch 15/50
21/21 - 0s - loss